# PREPROCESSING

### 1. Load All Library

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.classify import SklearnClassifier
from nltk.stem import LancasterStemmer, WordNetLemmatizer

import matplotlib.pyplot as plt

nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer
import re

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import preprocessor as p

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Load Dataset

In [3]:
data = pd.read_csv("./dataset/raw dataset/dataset_penyisihan_bdc_2024.csv",sep=';')

### 3. Data Assesment

In [4]:
data.info()
data.count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   object
dtypes: object(2)
memory usage: 78.2+ KB


text     5000
label    5000
dtype: int64

In [5]:
data.groupby('label').count()

,text
label,
Demografi,62
Ekonomi,367
Geografi,20
Ideologi,400
Pertahanan dan Keamanan,400
Politik,2972
Sosial Budaya,587
Sumber Daya Alam,192


### 4. Clean Data (Drop Duplicate)

In [6]:
data_clean = data.drop_duplicates()

### 5. Split Camel Case and Cleaning from Twitter Format 

In [7]:
def preprocess_tweet(data):
    text = data['preprocessed']
    text = p.clean(text)
    return text

def split_camel_case(text):
    return [re.sub(r'([a-z])([A-Z])', r'\1 \2', i) for i in text]

In [8]:
data['hastag'] = data['text'].apply(lambda x: re.findall(f"#(\w+)",x))
data_clean['preprocessed_hastag'] = data['hastag'].apply(split_camel_case)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2147847194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed_hastag'] = data['hastag'].apply(split_camel_case)


### 6. Lower The Text and Split The Text

In [9]:
data_clean['preprocessed_hastag'] = data_clean['preprocessed_hastag'].apply(lambda x: [i.lower() for i in x])
data_clean['preprocessed_hastag'] = data_clean['preprocessed_hastag'].apply(lambda x: ' '.join(x).split())

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\1267249655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed_hastag'] = data_clean['preprocessed_hastag'].apply(lambda x: [i.lower() for i in x])
C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\1267249655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed_hastag'] = data_clean['preprocessed_hastag'].apply(lambda x: ' '.join(x).split())


In [10]:
data_clean.head()

,text,label,preprocessed_hastag
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,"[indonesia, sentris, indonesia, hijau, 02melan..."
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,"[amin, miskinkan, koruptor]"
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,[]
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,[]
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,[]


### 7. Clean @ or (USERNAME) and # or (HASTAG) in Text

In [11]:
def clean_username(text):
    return re.sub(r'[@#]\S+', '', text)

data_clean['preprocessed'] = data_clean['text'].apply(lambda x : clean_username(x))

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2418940909.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed'] = data_clean['text'].apply(lambda x : clean_username(x))


### 8. Combine Hastag and Text

In [12]:
data_clean['preprocessed'] = data_clean['preprocessed'].apply(lambda x : x.split())
data_clean['preprocessed'] = data_clean['preprocessed'] + data_clean['preprocessed_hastag']
data_clean['preprocessed'] = data_clean['preprocessed'].apply(lambda x : ' '.join(x))

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\113627744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed'] = data_clean['preprocessed'].apply(lambda x : x.split())
C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\113627744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['preprocessed'] = data_clean['preprocessed'] + data_clean['preprocessed_hastag']
C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\113627744.py:3: SettingWithCopyWarning: 
A value is trying to be 

### 9. Lower, Delete Punctuation, and Delete Double Space

In [ ]:
import string

data_clean['lower'] = data_clean['preprocessed'].str.lower().str.replace("[^\w\s]","").str.replace("\s\s+","")

for index,row in data_clean.iterrows():
    row['lower'] = row['lower'].translate(str.maketrans('','',string.punctuation))
    data_clean['lower'][index] = row['lower']

### 10. Clean HTTPS and Number

In [14]:
data_clean['clean'] = data_clean['lower'].apply(lambda x: re.sub(r'\d+', '', x))
data_clean['clean'] = data_clean['lower'].apply(lambda x: re.sub(r'\b\w*https\w*\b', '', x))

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\4030215978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['clean'] = data_clean['lower'].apply(lambda x: re.sub(r'\d+', '', x))
C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\4030215978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['clean'] = data_clean['lower'].apply(lambda x: re.sub(r'\b\w*https\w*\b', '', x))


### 10. CHANGE NON FORMAL TO FORMAL

In [15]:
slang_list = pd.read_csv("./dataset/corpus dataset/colloquial-indonesian-lexicon.csv")
slang = slang_list['slang'].values.tolist()
formal = slang_list['formal'].values.tolist()
slangToformal = dict(zip(slang,formal))

formal_tokens = []
for index,row in data_clean.iterrows():
    temp = []
    for word in row['clean'].split():
        temp.append(slangToformal.get(word,word))
    res = " ".join(temp)
    formal_tokens.append(res)
data_clean["formal"] = formal_tokens

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\3932693264.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean["formal"] = formal_tokens


### 11. Cleaning Word that Start With Re (Retweet)

In [16]:
data_clean['formal'] = data_clean['formal'].str.replace(r're\s+\S+', '',regex=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2252865480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['formal'] = data_clean['formal'].str.replace(r're\s+\S+', '',regex=True)


## 13. Lematisasi

In [17]:
from nlp_id.lemmatizer import Lemmatizer

lemmatizer = Lemmatizer()
lemmatized=[]
for index, row in data_clean.iterrows():
    lemmatized.append(lemmatizer.lemmatize(row['formal']))

data_clean['lemmatized']=lemmatized

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\3671842305.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['lemmatized']=lemmatized


### 13. Stopword Removal

In [18]:
from nlp_id.stopword import StopWord

stopword = StopWord()
stopword_removed=[]

for index, row in data_clean.iterrows():
    stopword_removed.append(stopword.remove_stopword(row['lemmatized']))

data_clean['stopword_removed']=stopword_removed

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2645367154.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['stopword_removed']=stopword_removed


### 14. Tokenisasi

In [19]:
from nlp_id.tokenizer import Tokenizer
tokenizer = Tokenizer()

tokens_c=[]

for index, row in data_clean.iterrows():
    tokens = tokenizer.tokenize(row['stopword_removed'])
    tokens_c.append(tokens)

data_clean['tokens']=tokens_c

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2005421490.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['tokens']=tokens_c


### 15. Second Stopword Removal

In [20]:
#augment the stopwords with nonstandard twitter words
stopwords_set = set(stopwords.words("indonesian"))
stopwords_aug = {"ya","yak","iya","yg","ga","gak","gk","udh","sdh","udah","dah","nih","ini","deh","sih","dong","donk",
                 "sm","knp","utk","yaa","tdk","gini","gitu","bgt","gt","nya","kalo","cb","jg","jgn","gw","ge",
                 "sy","min","mas","mba","mbak","pak","kak","trus","trs","bs","bisa","aja","saja","no",
                 "w","g","gua","gue","emang","emg","wkwk","dr","kau","dg","gimana","apapun","apa",
                 "klo","yah","banget","pake","terus","krn","jadi","jd","mu","ku","si","hehe",
                 "tp","pa","lu","lo","lw","tw","tau","karna","kayak","ky","lg","untuk","tuk","dg","dgn"}
stopwords_all = stopwords_set.union(stopwords_aug)

data_clean['stopword_removed2'] = data_clean['tokens'].apply(lambda x: [item for item in x if item not in stopwords_all])

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\3561544505.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['stopword_removed2'] = data_clean['tokens'].apply(lambda x: [item for item in x if item not in stopwords_all])


### 16. Remove Digit in List

In [21]:
data_clean['stopword_removed2'] = data_clean['stopword_removed2'].apply(lambda x : [item for item in x if not item.isdigit()])

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\1636423664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['stopword_removed2'] = data_clean['stopword_removed2'].apply(lambda x : [item for item in x if not item.isdigit()])


### 17. Translate to Indonesia

In [22]:
from deep_translator import GoogleTranslator

def translate_to_indonesian(text):
    translated_text = GoogleTranslator(source='en', target='id').translate(text)
    return translated_text

indices_to_translate = [40, 415, 1646, 2973, 3767, 3721, 4736, 392,39,772,1121,1269,1787,306,2271,2344,17]

def conditional_translation(row, index):
    if index in indices_to_translate:
        return translate_to_indonesian(' '.join(row))
    else:
        return ' '.join(row)

In [23]:
data_clean['stopword_removed2'] = data_clean.apply(
    lambda row: conditional_translation(row['stopword_removed2'], row.name),
    axis=1)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\3102551940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['stopword_removed2'] = data_clean.apply(


### 18. Filter in KBBI Corpus

In [24]:
kbbi_corpus = pd.read_csv("./dataset/corpus dataset/kbbi.csv")
kbbi_corpus.drop_duplicates(inplace=True)
list_kbbi = kbbi_corpus['kata'].to_list()

def hapus_kata_non_sastrawi(kata_list):
    kata_list = kata_list.split()
    return [kata for kata in kata_list if kata in list_kbbi]

In [25]:
# Terapkan fungsi ke kolom stopword_removed2
data_clean['KBBI'] = data_clean['stopword_removed2'].apply(hapus_kata_non_sastrawi)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2736237675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['KBBI'] = data_clean['stopword_removed2'].apply(hapus_kata_non_sastrawi)


### 19. Remove Word that Just Consist of 2 Letter and Replace Milu to Pemilu

In [26]:
def clear_list(clear2List):
    # Menghapus elemen dengan panjang kurang dari atau sama dengan 2 karakter
    list_karakter = [item for item in clear2List if len(item) > 2]

    # Mengganti semua kemunculan "milu" menjadi "pemilu" di dalam list
    for i in range(len(list_karakter)):
        list_karakter[i] = list_karakter[i].replace("milu", "pemilu")
    
    return list_karakter

data_clean['cleaned_KBBI'] = data_clean['KBBI'].apply(clear_list)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\1707700021.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['cleaned_KBBI'] = data_clean['KBBI'].apply(clear_list)


### 20. Optional Make it To Unique Words

In [27]:
data_clean['unique_teks'] = data_clean['cleaned_KBBI'].apply(lambda x : ' '.join(set(x)))

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\4079798604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['unique_teks'] = data_clean['cleaned_KBBI'].apply(lambda x : ' '.join(set(x)))


### 21. Drop Nan Result

In [28]:
data_clean.dropna(inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2410125117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.dropna(inplace=True)


In [29]:
data_clean.head()

,text,label,preprocessed_hastag,preprocessed,lower,clean,formal,lemmatized,stopword_removed,tokens,stopword_removed2,KBBI,cleaned_KBBI,unique_teks
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,"[indonesia, sentris, indonesia, hijau, 02melan...",Kunjungan Prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjung prabowo ini untuk resmi dan serah proy...,kunjung prabowo resmi serah proyek bantu air b...,"[kunjung, prabowo, resmi, serah, proyek, bantu...",kunjung prabowo resmi serah proyek bantu air b...,"[kunjung, prabowo, resmi, serah, proyek, bantu...","[kunjung, prabowo, resmi, serah, proyek, bantu...",titik anak kunjung hijau bantu prabowo emas mu...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,"[amin, miskinkan, koruptor]",RT Anies dapat tepuk tangan meriah saat jadi R...,rt anies dapat tepuk tangan meriah saat jadi r...,rt anies dapat tepuk tangan meriah saat jadi r...,rt anies dapat tepuk tangan meriah saat jadi r...,rt anies dapat tepuk tangan riah saat jadi rek...,rt anies tepuk tangan riah rektor wajib mata k...,"[rt, anies, tepuk, tangan, riah, rektor, wajib...",rt anies tepuk tangan riah rektor wajib mata k...,"[rt, anies, tepuk, tangan, riah, rektor, wajib...","[anies, tepuk, tangan, riah, rektor, wajib, ma...",koruptor antikorupsi mata tangan tepuk putus r...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,[],"emng bener sih, pendukung 01 ada yg goblok, be...",emng bener sih pendukung 01 ada yg goblok begi...,emng bener sih pendukung 01 ada yg goblok begi...,memang benar sih pendukung 01 ada yang goblok ...,memang benar sih dukung 01 ada yang goblok beg...,dukung 01 goblok dukung 02 ridwan kamil skema ...,"[dukung, 01, goblok, dukung, 02, ridwan, kamil...",dukung goblok dukung ridwan kamil skema mayori...,"[dukung, goblok, dukung, kamil, skema, mayorit...","[dukung, goblok, dukung, kamil, skema, mayorit...",pilih rendah kamil skema mayoritas goblok duku...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,[],RT Sewaktu anies bersikap kritis ke kinerja pa...,rt sewaktu anies bersikap kritis ke kinerja pa...,rt sewaktu anies bersikap kritis ke kinerja pa...,rt sewaktu anies bersikap kritis ke kinerja pa...,rt waktu anies sikap kritis ke kerja pak prabo...,rt anies sikap kritis kerja prabowo anggap sop...,"[rt, anies, sikap, kritis, kerja, prabowo, ang...",rt anies sikap kritis kerja prabowo anggap sop...,"[rt, anies, sikap, kritis, kerja, prabowo, ang...","[anies, sikap, kritis, kerja, prabowo, anggap,...",kritis hormat tengil standar anggap prabowo gi...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,[],Anies Baswedan Harap ASN termasuk TNI dan Polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn masuk tni dan polri p...,anies baswedan harap asn masuk tni polri pegan...,"[anies, baswedan, harap, asn, masuk, tni, polr...",anies baswedan harap asn masuk tni polri pegan...,"[anies, harap, masuk, tni, polri, pegang, sump...","[anies, harap, masuk, tni, polri, pegang, sump...",pemilu sumpah tni masuk harap pegang anies polri


In [30]:
data.groupby('label').count()

,text,hastag
label,,
Demografi,62,62
Ekonomi,367,367
Geografi,20,20
Ideologi,400,400
Pertahanan dan Keamanan,400,400
Politik,2972,2972
Sosial Budaya,587,587
Sumber Daya Alam,192,192


In [31]:
data_clean['TEKS'] = data_clean['cleaned_KBBI'].apply(lambda x : ' '.join(x))

C:\Users\Asus\AppData\Local\Temp\ipykernel_23600\2170106849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['TEKS'] = data_clean['cleaned_KBBI'].apply(lambda x : ' '.join(x))


In [ ]:
data_clean.to_csv("./dataset/processed_dataset/processed_dataset.csv",index=False)

In [32]:
data_clean[['stopword_removed2','label']].dropna().to_csv("latih_lagi.csv",index=False)